In [1]:
!pip install datasets transformers[torch] tokenizers seqeval -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 13.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 32.4 MB/s eta 0:00:00


In [2]:
import datasets
import numpy as np
from transformers import BertTokenizerFast
from transformers import DataCollatorForTokenClassification
from transformers import AutoModelForTokenClassification
from transformers import TrainingArguments, Trainer
import json

In [3]:
metric = datasets.load_metric("seqeval")
conll2003 = datasets.load_dataset("conll2003")
label_list = conll2003["train"].features["ner_tags"].feature.names

<ipython-input-3-55331d1de86d>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = datasets.load_metric("seqeval")


Generating train split:   0%|          | 0/14041 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3250 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [4]:
def RmvColumns(data, columns_to_remove = ['id', 'pos_tags', 'chunk_tags']):
    data = data.remove_columns(columns_to_remove)
    return data

def tokenize_and_align_labels(examples, label_all_tokens=True):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)
    labels = []
    for i, label in enumerate(examples["ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)
        # word_ids() => Return a list mapping the tokens
        # to their actual word in the initial sentence.
        # It Returns a list indicating the word corresponding to each token.
        previous_word_idx = None
        label_ids = []
        # Special tokens like `` and `<\s>` are originally mapped to None
        # We need to set the label to -100 so they are automatically ignored in the loss function.
        for word_idx in word_ids:
            if word_idx is None:
                # set –100 as the label for these special tokens
                label_ids.append(-100)
            # For the other tokens in a word, we set the label to either the current label or -100, depending on
            # the label_all_tokens flag.
            elif word_idx != previous_word_idx:
                # if current word_idx is != prev then its the most regular case
                # and add the corresponding token
                label_ids.append(label[word_idx])
            else:
                # to take care of sub-words which have the same word_idx
                # set -100 as well for them, but only if label_all_tokens == False
                label_ids.append(label[word_idx] if label_all_tokens else -100)
                # mask the subword representations after the first subword

            previous_word_idx = word_idx
        labels.append(label_ids)
    tokenized_inputs["labels"] = labels
    return tokenized_inputs

def compute_metrics(eval_preds):
    pred_logits, labels = eval_preds

    pred_logits = np.argmax(pred_logits, axis=2)
    # the logits and the probabilities are in the same order,
    # so we don’t need to apply the softmax

    # We remove all the values where the label is -100
    predictions = [
        [label_list[eval_preds] for (eval_preds, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(pred_logits, labels)
    ]

    true_labels = [
      [label_list[l] for (eval_preds, l) in zip(prediction, label) if l != -100]
       for prediction, label in zip(pred_logits, labels)
   ]
    results = metric.compute(predictions=predictions, references=true_labels)

    return {
            "precision": results["overall_precision"],
            "recall": results["overall_recall"],
            "f1": results["overall_f1"],
            "accuracy": results["overall_accuracy"],
           }

def Train(tokenized_datasets, model, tokenizer, data_collator):
    args = TrainingArguments(
                "test-ner",
                evaluation_strategy = "epoch",
                learning_rate=2e-5,
                per_device_train_batch_size=16,
                per_device_eval_batch_size=16,
                num_train_epochs=3,
                weight_decay=0.01,
          )

    trainer = Trainer(
                model,
                args,
                train_dataset=tokenized_datasets["train"],
                eval_dataset=tokenized_datasets["validation"],
                data_collator=data_collator,
                tokenizer=tokenizer,
                compute_metrics=compute_metrics
          )

    trainer.train()
    return model

In [5]:
conll2003 = RmvColumns(conll2003)
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=9)
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
data_collator = DataCollatorForTokenClassification(tokenizer)
tokenized_datasets = conll2003.map(tokenize_and_align_labels, batched=True)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/14041 [00:00<?, ? examples/s]

Map:   0%|          | 0/3250 [00:00<?, ? examples/s]

Map:   0%|          | 0/3453 [00:00<?, ? examples/s]

In [6]:
conll2003['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0]}

In [7]:
tokenized_datasets['train'][0]

{'tokens': ['EU',
  'rejects',
  'German',
  'call',
  'to',
  'boycott',
  'British',
  'lamb',
  '.'],
 'ner_tags': [3, 0, 7, 0, 0, 0, 7, 0, 0],
 'input_ids': [101,
  7327,
  19164,
  2446,
  2655,
  2000,
  17757,
  2329,
  12559,
  1012,
  102],
 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 3, 0, 7, 0, 0, 0, 7, 0, 0, -100]}

In [8]:
model = Train(tokenized_datasets, model, tokenizer, data_collator)

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.219700,0.065253,0.915741,0.931312,0.923461,0.982239
2,0.045600,0.057999,0.933474,0.941828,0.937632,0.985257
3,0.026900,0.059774,0.935934,0.944625,0.940259,0.985813


In [9]:
model.save_pretrained("ner_model")
tokenizer.save_pretrained("tokenizer")

('tokenizer/tokenizer_config.json',
 'tokenizer/special_tokens_map.json',
 'tokenizer/vocab.txt',
 'tokenizer/added_tokens.json',
 'tokenizer/tokenizer.json')

In [23]:
from transformers import pipeline
from prettytable import PrettyTable

label_map = {'LABEL_0':'O', 'LABEL_1':'B-PER', 'LABEL_2':'I-PER', 'LABEL_3':'B-ORG', 'LABEL_4':'I-ORG', 'LABEL_5':'B-LOC', 'LABEL_6':'I-LOC', 'LABEL_7':'B-MISC', 'LABEL_8':'I-MISC'}
model = AutoModelForTokenClassification.from_pretrained("ner_model")
nlp = pipeline("ner", model=model, tokenizer=tokenizer)

In [26]:
def DecodeOutput(data, model = model, nlp = nlp, label_map = label_map):
    data = nlp(data)
    data = [{'word': item['word'], 'entity': item['entity']} for item in data]
    data = [{'word': item['word'], 'entity': label_map.get(item['entity'], item['entity'])} for item in data]
    table = PrettyTable(['Word', 'Entity'])

    for item in data:
        table.add_row([item['word'], item['entity']])

    print(table)
    return data

example = "Bill Gates is the Founder of Microsoft"
example = DecodeOutput(example)

+-----------+--------+
|    Word   | Entity |
+-----------+--------+
|    bill   | B-PER  |
|   gates   | I-PER  |
|     is    |   O    |
|    the    |   O    |
|  founder  |   O    |
|     of    |   O    |
| microsoft | B-ORG  |
+-----------+--------+
